In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from anticipation.config import *
from anticipation.vocab import *

import annotation_mapping as am

from anticipation.convert import midi_to_events, midi_to_compound
from train.midi_preprocess import convert_midi

from anticipation.tokenize import tokenize2, maybe_tokenize


In [2]:
df = pd.read_csv('asap-dataset-master/metadata.csv')

In [3]:
df.head()

,composer,title,folder,xml_score,midi_score,midi_performance,performance_annotations,midi_score_annotations,maestro_midi_performance,maestro_audio_performance,start,end,audio_performance
0,Bach,Fugue_bwv_846,Bach/Fugue/bwv_846,Bach/Fugue/bwv_846/xml_score.musicxml,Bach/Fugue/bwv_846/midi_score.mid,Bach/Fugue/bwv_846/Shi05M.mid,Bach/Fugue/bwv_846/Shi05M_annotations.txt,Bach/Fugue/bwv_846/midi_score_annotations.txt,{maestro}/2006/MIDI-Unprocessed_19_R1_2006_01-...,{maestro}/2006/MIDI-Unprocessed_19_R1_2006_01-...,144.567708,NaN,Bach/Fugue/bwv_846/Shi05M.wav
1,Bach,Fugue_bwv_848,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/xml_score.musicxml,Bach/Fugue/bwv_848/midi_score.mid,Bach/Fugue/bwv_848/Denisova06M.mid,Bach/Fugue/bwv_848/Denisova06M_annotations.txt,Bach/Fugue/bwv_848/midi_score_annotations.txt,{maestro}/2008/MIDI-Unprocessed_02_R1_2008_01-...,{maestro}/2008/MIDI-Unprocessed_02_R1_2008_01-...,69.317708,NaN,Bach/Fugue/bwv_848/Denisova06M.wav
2,Bach,Fugue_bwv_848,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/xml_score.musicxml,Bach/Fugue/bwv_848/midi_score.mid,Bach/Fugue/bwv_848/Lee01M.mid,Bach/Fugue/bwv_848/Lee01M_annotations.txt,Bach/Fugue/bwv_848/midi_score_annotations.txt,{maestro}/2011/MIDI-Unprocessed_08_R1_2011_MID...,{maestro}/2011/MIDI-Unprocessed_08_R1_2011_MID...,74.795573,NaN,Bach/Fugue/bwv_848/Lee01M.wav
3,Bach,Fugue_bwv_848,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/xml_score.musicxml,Bach/Fugue/bwv_848/midi_score.mid,Bach/Fugue/bwv_848/LeeSH01M.mid,Bach/Fugue/bwv_848/LeeSH01M_annotations.txt,Bach/Fugue/bwv_848/midi_score_annotations.txt,{maestro}/2017/MIDI-Unprocessed_049_PIANO049_M...,{maestro}/2017/MIDI-Unprocessed_049_PIANO049_M...,83.644792,NaN,Bach/Fugue/bwv_848/LeeSH01M.wav
4,Bach,Fugue_bwv_848,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/xml_score.musicxml,Bach/Fugue/bwv_848/midi_score.mid,Bach/Fugue/bwv_848/Lin04M.mid,Bach/Fugue/bwv_848/Lin04M_annotations.txt,Bach/Fugue/bwv_848/midi_score_annotations.txt,{maestro}/2008/MIDI-Unprocessed_10_R1_2008_01-...,{maestro}/2008/MIDI-Unprocessed_10_R1_2008_01-...,75.373698,NaN,Bach/Fugue/bwv_848/Lin04M.wav


In [4]:
BASE = "./asap-dataset-master/"

In [5]:
sample = df.iloc[0]
sample

composer                                                                  Bach
title                                                            Fugue_bwv_846
folder                                                      Bach/Fugue/bwv_846
xml_score                                Bach/Fugue/bwv_846/xml_score.musicxml
midi_score                                   Bach/Fugue/bwv_846/midi_score.mid
midi_performance                                 Bach/Fugue/bwv_846/Shi05M.mid
performance_annotations              Bach/Fugue/bwv_846/Shi05M_annotations.txt
midi_score_annotations           Bach/Fugue/bwv_846/midi_score_annotations.txt
maestro_midi_performance     {maestro}/2006/MIDI-Unprocessed_19_R1_2006_01-...
maestro_audio_performance    {maestro}/2006/MIDI-Unprocessed_19_R1_2006_01-...
start                                                               144.567708
end                                                                        NaN
audio_performance                                Bac

In [6]:
datafiles = []

for i, row in df.iterrows():
    file1 = BASE + row['midi_performance']
    file2 = BASE + row['midi_score']
    file3 = BASE + row['performance_annotations']
    file4 = BASE + row['midi_score_annotations']

    datafiles.append((file1, file2, file3, file4))

indices = np.random.choice(1000, size=213, replace=False)
validation_indices, test_indices = indices[:107], indices[107:]

train_datafiles = [datafiles[i] for i in range(len(datafiles)) if i not in indices]
validation_datafiles = [datafiles[i] for i in validation_indices]
test_datafiles = [datafiles[i] for i in test_indices]

In [7]:
tokenize2(test_datafiles, output='test_output.txt')

Number of annotations in ./asap-dataset-master/Rachmaninoff/Preludes_op_32/10/midi_score_annotations.txt and ./asap-dataset-master/Rachmaninoff/Preludes_op_32/10/FONG08_annotations.txt do not match.
Proceeding with the first 192 annotations.


Number of annotations in ./asap-dataset-master/Chopin/Sonata_2/3rd/midi_score_annotations.txt and ./asap-dataset-master/Chopin/Sonata_2/3rd/KaszoS15_annotations.txt do not match.
Proceeding with the first 367 annotations.


Number of annotations in ./asap-dataset-master/Beethoven/Piano_Sonatas/3-1/midi_score_annotations.txt and ./asap-dataset-master/Beethoven/Piano_Sonatas/3-1/LUO01_annotations.txt do not match.
Proceeding with the first 1056 annotations.


Number of annotations in ./asap-dataset-master/Beethoven/Piano_Sonatas/26-3/midi_score_annotations.txt and ./asap-dataset-master/Beethoven/Piano_Sonatas/26-3/HONG05M_annotations.txt do not match.
Proceeding with the first 391 annotations.


#0: 100%|██████████| 106/106 [01:19<00:00,  1.33it/s]


(1489, 2057, 0, 0, 0, 695, 42)

In [23]:
events = maybe_tokenize(midi_to_compound(file2))[0]

In [25]:
for t, d, n in zip(events[0::3],events[1::3],events[2::3]):
    print(t, d, n)

25 10025 11060
50 10025 11062
75 10025 11064
100 10037 11065
138 10006 11067
144 10006 11065
150 10025 11064
175 10025 11069
200 10025 11062
225 10037 11067
262 10012 11069
275 10012 11067
288 10012 11065
300 10012 11064
312 10012 11065
325 10025 11067
325 10012 11064
338 10012 11062
350 10025 11069
350 10012 11060
362 10012 11062
375 10025 11071
375 10012 11060
388 10012 11059
400 10037 11072
400 10025 11057
425 10025 11066
438 10006 11074
444 10006 11072
450 10025 11071
450 10075 11067
475 10025 11076
500 10025 11069
525 10037 11074
525 10012 11066
538 10012 11064
550 10025 11066
562 10012 11076
575 10012 11074
575 10025 11062
588 10012 11072
600 10012 11071
600 10012 11067
612 10012 11067
625 10012 11069
625 10025 11065
625 10025 11055
638 10012 11071
650 10012 11072
650 10025 11064
650 10025 11057
662 10012 11071
675 10012 11072
675 10025 11062
675 10025 11059
688 10012 11074
700 10012 11076
700 10000 11060
700 10037 11060
712 10012 11074
725 10012 11076
738 10012 11078
738 10006 1

In [31]:
CONTROL_OFFSET

27513

In [9]:
l = [1,2,3]
l.extend((1,2,3))
l

[1, 2, 3, 1, 2, 3]